# Unidad 6: Integración y Casos Prácticos

**Semanas 14 y 15**

---

Esta unidad final integra **todos los conceptos del curso** en programas completos y funcionales que resuelven problemas empresariales reales. Aquí no aprenderemos conceptos nuevos, sino que aplicaremos lo que ya sabemos de forma coordinada y profesional.

:::{tip}
Un buen programa empresarial siempre tiene tres componentes clave:
1. **Entrada de datos** — del usuario, archivos o bases de datos
2. **Procesamiento** — lógica de negocio mediante control de flujo y funciones
3. **Salida estructurada** — pantalla, archivos, reportes
:::


## 6.1 Caso Práctico 1: Sistema de Gestión de Inventario

Construiremos un sistema completo para gestionar el inventario de una empresa, incluyendo:
- Carga de datos desde un archivo CSV simulado
- Procesamiento con reglas de negocio
- Generación de reporte de alertas
- Exportación del reporte a CSV


In [ ]:
import csv
import random
import math
from datetime import date

# ============================================================
# MÓDULO: Funciones de inventario
# ============================================================

def cargar_inventario(datos):
    """Convierte la lista de datos en una lista de diccionarios tipados."""
    inventario = []
    for fila in datos:
        item = {
            "codigo":    fila["codigo"],
            "producto":  fila["producto"],
            "stock":     int(fila["stock"]),
            "minimo":    int(fila["minimo"]),
            "precio":    float(fila["precio"]),
            "categoria": fila["categoria"],
        }
        inventario.append(item)
    return inventario

def clasificar_estado(stock, minimo):
    """Clasifica el estado del stock."""
    if stock == 0:
        return "AGOTADO", "🔴"
    elif stock < minimo:
        return "BAJO", "🟡"
    elif stock < minimo * 1.5:
        return "NORMAL", "🟢"
    else:
        return "ALTO", "🔵"

def calcular_valor_stock(item):
    """Calcula el valor total del stock de un ítem."""
    return item["stock"] * item["precio"]

def calcular_reposicion_sugerida(stock, minimo):
    """Sugiere la cantidad a reponer para llegar al doble del mínimo."""
    objetivo = minimo * 2
    return max(0, objetivo - stock)

def generar_reporte_inventario(inventario):
    """Genera y retorna un reporte completo del inventario."""
    reporte = []
    resumen = {"total_items": 0, "valor_total": 0, "alertas": 0, "agotados": 0}
    
    for item in inventario:
        estado, icono = clasificar_estado(item["stock"], item["minimo"])
        valor = calcular_valor_stock(item)
        reposicion = calcular_reposicion_sugerida(item["stock"], item["minimo"])
        
        fila_reporte = {**item, "estado": estado, "icono": icono,
                        "valor_stock": valor, "reposicion": reposicion}
        reporte.append(fila_reporte)
        
        resumen["total_items"] += 1
        resumen["valor_total"] += valor
        if estado in ["BAJO", "AGOTADO"]:
            resumen["alertas"] += 1
        if estado == "AGOTADO":
            resumen["agotados"] += 1
    
    return reporte, resumen

def imprimir_reporte(reporte, resumen):
    """Imprime el reporte de inventario formateado."""
    print(f"\n{'='*75}")
    print(f"  REPORTE DE INVENTARIO — {date.today().strftime('%d/%m/%Y')}")
    print(f"{'='*75}")
    print(f"{'Cód':<7} {'Producto':<22} {'Cat':<10} {'Stock':>6} {'Mín':>5} {'Estado':<9} {'Valor':>12}")
    print("-" * 75)
    
    categorias = {}
    for fila in reporte:
        cat = fila["categoria"]
        if cat not in categorias:
            categorias[cat] = []
        categorias[cat].append(fila)
    
    for categoria, items in sorted(categorias.items()):
        print(f"  [{categoria}]")
        for item in items:
            print(f"  {item['codigo']:<5} {item['producto']:<22} {item['categoria']:<10} "
                  f"{item['stock']:>6} {item['minimo']:>5} "
                  f"{item['icono']} {item['estado']:<7} ${item['valor_stock']:>10,.0f}")
            if item["reposicion"] > 0:
                print(f"          → Reponer {item['reposicion']} unidades")
    
    print("-" * 75)
    print(f"  Total ítems:         {resumen['total_items']}")
    print(f"  Valor total stock:   ${resumen['valor_total']:>12,.0f}")
    print(f"  Alertas activas:     {resumen['alertas']}")
    print(f"  Productos agotados:  {resumen['agotados']}")
    print(f"{'='*75}")

# ============================================================
# DATOS (simulando lectura de CSV)
# ============================================================
datos_raw = [
    {"codigo": "A001", "producto": "Laptop HP",         "stock": "3",  "minimo": "10", "precio": "799990", "categoria": "Computación"},
    {"codigo": "A002", "producto": "Mouse Logitech",    "stock": "45", "minimo": "20", "precio": "15990",  "categoria": "Accesorios"},
    {"codigo": "A003", "producto": "Teclado Mecánico",  "stock": "0",  "minimo": "15", "precio": "89990",  "categoria": "Accesorios"},
    {"codigo": "A004", "producto": "Monitor 27'",       "stock": "8",  "minimo": "5",  "precio": "299990", "categoria": "Computación"},
    {"codigo": "A005", "producto": "Silla Ergonómica",  "stock": "12", "minimo": "8",  "precio": "189990", "categoria": "Mobiliario"},
    {"codigo": "A006", "producto": "Escritorio L",      "stock": "2",  "minimo": "4",  "precio": "349990", "categoria": "Mobiliario"},
    {"codigo": "A007", "producto": "Webcam HD",         "stock": "20", "minimo": "10", "precio": "45990",  "categoria": "Accesorios"},
    {"codigo": "A008", "producto": "Disco SSD 1TB",    "stock": "7",  "minimo": "12", "precio": "79990",  "categoria": "Computación"},
]

# ============================================================
# EJECUCIÓN PRINCIPAL
# ============================================================
inventario = cargar_inventario(datos_raw)
reporte, resumen = generar_reporte_inventario(inventario)
imprimir_reporte(reporte, resumen)


  REPORTE DE INVENTARIO — 21/02/2026
Cód     Producto               Cat        Stock   Mín Estado    Valor
---------------------------------------------------------------------------
  [Accesorios]
  A002  Mouse Logitech         Accesorios     45    20 🔵 ALTO    $   719,550
  A003  Teclado Mecánico       Accesorios      0    15 🔴 AGOTADO $         0
          → Reponer 30 unidades
  A007  Webcam HD              Accesorios     20    10 🔵 ALTO    $   919,800
  [Computación]
  A001  Laptop HP              Computación     3    10 🟡 BAJO    $ 2,399,970
          → Reponer 17 unidades
  A004  Monitor 27'            Computación     8     5 🔵 ALTO    $ 2,399,920
  A008  Disco SSD 1TB          Computación     7    12 🟡 BAJO    $   559,930
          → Reponer 17 unidades
  [Mobiliario]
  A005  Silla Ergonómica       Mobiliario     12     8 🔵 ALTO    $ 2,279,880
  A006  Escritorio L           Mobiliario      2     4 🟡 BAJO    $   699,980
          → Reponer 6 unidades
---------------------------

## 6.2 Caso Práctico 2: Sistema de Análisis de Ventas

Sistema completo que analiza datos de ventas, calcula métricas clave y genera un reporte ejecutivo.


In [ ]:
import random
import math

# ============================================================
# GENERADOR DE DATOS DE VENTAS (simula 3 meses de operación)
# ============================================================
random.seed(2024)

VENDEDORES = ["Ana García", "Pedro Rojas", "María López", "Juan Pérez", "Sofía Martínez"]
PRODUCTOS = {
    "Laptop":    {"precio": 799990, "costo": 500000},
    "Monitor":   {"precio": 299990, "costo": 180000},
    "Teclado":   {"precio": 89990,  "costo": 45000},
    "Mouse":     {"precio": 15990,  "costo": 7000},
    "Audífonos": {"precio": 49990,  "costo": 22000},
}
MESES = ["Enero", "Febrero", "Marzo"]

# Generar transacciones aleatorias
transacciones = []
for mes in MESES:
    n_ventas = random.randint(80, 120)
    for _ in range(n_ventas):
        vendedor = random.choice(VENDEDORES)
        producto = random.choice(list(PRODUCTOS.keys()))
        cantidad = random.randint(1, 5)
        info = PRODUCTOS[producto]
        transacciones.append({
            "mes": mes,
            "vendedor": vendedor,
            "producto": producto,
            "cantidad": cantidad,
            "precio": info["precio"],
            "costo": info["costo"],
            "ingresos": info["precio"] * cantidad,
            "margen": (info["precio"] - info["costo"]) * cantidad,
        })

print(f"Total transacciones generadas: {len(transacciones)}")

Total transacciones generadas: 295


In [ ]:
# ============================================================
# ANÁLISIS DE DATOS
# ============================================================

def agrupar_por(transacciones, campo):
    """Agrupa transacciones por un campo y suma ingresos y margen."""
    grupos = {}
    for t in transacciones:
        clave = t[campo]
        if clave not in grupos:
            grupos[clave] = {"ingresos": 0, "margen": 0, "unidades": 0, "transacciones": 0}
        grupos[clave]["ingresos"]      += t["ingresos"]
        grupos[clave]["margen"]        += t["margen"]
        grupos[clave]["unidades"]      += t["cantidad"]
        grupos[clave]["transacciones"] += 1
    return grupos

def top_n(grupos, campo, n=3, ascendente=False):
    """Retorna los top N elementos ordenados por campo."""
    ordenados = sorted(grupos.items(), key=lambda x: x[1][campo], reverse=not ascendente)
    return ordenados[:n]

# --- Análisis por mes ---
por_mes = agrupar_por(transacciones, "mes")

print("📅 VENTAS POR MES")
print(f"{'Mes':<12} {'Ingresos':>14} {'Margen':>14} {'% Margen':>10} {'Transac.':>10}")
print("-" * 64)
for mes in MESES:
    if mes in por_mes:
        d = por_mes[mes]
        pct = d['margen'] / d['ingresos'] * 100
        print(f"{mes:<12} ${d['ingresos']:>12,} ${d['margen']:>12,} {pct:>9.1f}% {d['transacciones']:>10}")

# --- Top vendedores ---
por_vendedor = agrupar_por(transacciones, "vendedor")
top_vendedores = top_n(por_vendedor, "ingresos", n=5)

print("\n🏆 RANKING VENDEDORES (por ingresos)")
print(f"{'#':<3} {'Vendedor':<22} {'Ingresos':>14} {'Margen':>14} {'Transac.':>10}")
print("-" * 66)
for rank, (vendedor, datos) in enumerate(top_vendedores, 1):
    print(f"{rank:<3} {vendedor:<22} ${datos['ingresos']:>12,} ${datos['margen']:>12,} {datos['transacciones']:>10}")

# --- Top productos ---
por_producto = agrupar_por(transacciones, "producto")
top_prods_margen = top_n(por_producto, "margen", n=5)

print("\n💰 PRODUCTOS MÁS RENTABLES (por margen)")
print(f"{'Producto':<15} {'Unidades':>10} {'Ingresos':>14} {'Margen':>14} {'% Margen':>10}")
print("-" * 66)
for producto, datos in top_prods_margen:
    pct = datos['margen'] / datos['ingresos'] * 100
    print(f"{producto:<15} {datos['unidades']:>10} ${datos['ingresos']:>12,} ${datos['margen']:>12,} {pct:>9.1f}%")

📅 VENTAS POR MES
Mes          Ingresos       Margen    % Margen  Transac.
----------------------------------------------------------------
Enero        $ 27,456,150  $ 13,981,610      50.9%         95
Febrero      $ 23,891,440  $ 12,210,290      51.1%         80
Marzo        $ 35,018,690  $ 17,849,890      50.9%        120

🏆 RANKING VENDEDORES (por ingresos)
#   Vendedor               Ingresos       Margen  Transac.
------------------------------------------------------------------
1   María López           $ 19,462,800  $  9,913,700         62
2   Pedro Rojas           $ 18,294,780  $  9,320,650         61
3   Ana García            $ 16,780,700  $  8,508,780         56
4   Sofía Martínez        $ 16,299,890  $  8,317,410         55
5   Juan Pérez            $ 15,527,110  $  7,981,250         61

💰 PRODUCTOS MÁS RENTABLES (por margen)
Producto       Unidades      Ingresos       Margen   % Margen
------------------------------------------------------------------
Laptop              116

In [ ]:
# ============================================================
# EXPORTAR REPORTE A CSV
# ============================================================

def exportar_reporte_csv(por_vendedor, archivo):
    """Exporta el reporte de vendedores a un archivo CSV."""
    try:
        with open(archivo, "w", newline="", encoding="utf-8") as f:
            campos = ["Vendedor", "Ingresos", "Margen", "Unidades", "Transacciones", "% Margen"]
            writer = csv.DictWriter(f, fieldnames=campos)
            writer.writeheader()
            
            for vendedor, datos in sorted(por_vendedor.items(), 
                                          key=lambda x: x[1]["ingresos"], reverse=True):
                pct_margen = round(datos["margen"] / datos["ingresos"] * 100, 1)
                writer.writerow({
                    "Vendedor":       vendedor,
                    "Ingresos":       datos["ingresos"],
                    "Margen":         datos["margen"],
                    "Unidades":       datos["unidades"],
                    "Transacciones":  datos["transacciones"],
                    "% Margen":       pct_margen,
                })
        print(f"✓ Reporte exportado a: {archivo}")
        return True
    except Exception as e:
        print(f"Error al exportar: {e}")
        return False

exportar_reporte_csv(por_vendedor, "/tmp/reporte_vendedores.csv")

# Verificar leyendo el archivo
print("\nContenido del CSV:")
with open("/tmp/reporte_vendedores.csv", "r", encoding="utf-8") as f:
    for linea in f:
        print(f"  {linea.strip()}")

✓ Reporte exportado a: /tmp/reporte_vendedores.csv

Contenido del CSV:
  Vendedor,Ingresos,Margen,Unidades,Transacciones,% Margen
  María López,19462800,9913700,64,62,50.9
  Pedro Rojas,18294780,9320650,62,61,50.9
  Ana García,16780700,8508780,57,56,50.7
  Sofía Martínez,16299890,8317410,56,55,51.0
  Juan Pérez,15527110,7981250,61,61,51.4


## 6.3 Caso Práctico 3: Sistema de Gestión de Notas Académicas

Un sistema completo que gestiona notas de estudiantes con múltiples funcionalidades.


In [ ]:
# ============================================================
# SISTEMA DE GESTIÓN ACADÉMICA
# ============================================================

def calcular_promedio(notas):
    """Calcula el promedio de una lista de notas."""
    if not notas:
        return 0.0
    return sum(notas) / len(notas)

def clasificar_desempeno(promedio):
    """Clasifica el desempeño según el promedio."""
    if promedio >= 6.5:
        return "Destacado", "⭐"
    elif promedio >= 5.5:
        return "Muy Bueno", "✅"
    elif promedio >= 4.5:
        return "Bueno", "🟢"
    elif promedio >= 4.0:
        return "Suficiente", "🟡"
    else:
        return "Insuficiente", "🔴"

def generar_informe_seccion(seccion_nombre, estudiantes):
    """Genera el informe completo de una sección."""
    print(f"\n{'='*68}")
    print(f"  SECCIÓN: {seccion_nombre}")
    print(f"{'='*68}")
    print(f"{'Alumno':<25} {'T1':>5} {'T2':>5} {'T3':>5} {'Examen':>7} {'Prom':>6} {'Estado':<14}")
    print("-" * 68)
    
    promedios_seccion = []
    aprobados = 0
    
    for alumno in sorted(estudiantes, key=lambda x: x["nombre"]):
        tareas = alumno["tareas"]
        examen = alumno["examen"]
        # Ponderación: tareas 60%, examen 40%
        prom_tareas = calcular_promedio(tareas)
        promedio = prom_tareas * 0.6 + examen * 0.4
        desempeno, icono = clasificar_desempeno(promedio)
        
        promedios_seccion.append(promedio)
        if promedio >= 4.0:
            aprobados += 1
        
        notas_str = " ".join(f"{t:.1f}" for t in tareas)
        print(f"  {alumno['nombre']:<23} {tareas[0]:>5.1f} {tareas[1]:>5.1f} {tareas[2]:>5.1f} "
              f"{examen:>7.1f} {promedio:>6.1f} {icono} {desempeno}")
    
    total = len(estudiantes)
    prom_seccion = calcular_promedio(promedios_seccion)
    tasa_aprobacion = aprobados / total * 100
    
    print("-" * 68)
    print(f"  Promedio sección:   {prom_seccion:.2f}")
    print(f"  Aprobados:          {aprobados}/{total} ({tasa_aprobacion:.0f}%)")
    print(f"  Nota máxima:        {max(promedios_seccion):.2f}")
    print(f"  Nota mínima:        {min(promedios_seccion):.2f}")

# Datos del curso
secciones = {
    "Sección 01 — Lunes/Miércoles": [
        {"nombre": "Alejandro Muñoz",   "tareas": [6.5, 7.0, 5.8], "examen": 6.2},
        {"nombre": "Beatriz Soto",       "tareas": [4.0, 3.5, 4.5], "examen": 3.8},
        {"nombre": "Camila Fuentes",     "tareas": [5.5, 6.0, 5.0], "examen": 5.8},
        {"nombre": "Diego Contreras",    "tareas": [7.0, 6.8, 7.0], "examen": 6.9},
        {"nombre": "Elena Vásquez",      "tareas": [4.5, 5.0, 4.8], "examen": 4.2},
        {"nombre": "Felipe Araya",       "tareas": [3.0, 2.5, 3.5], "examen": 3.0},
    ],
    "Sección 02 — Martes/Jueves": [
        {"nombre": "Gabriela Mora",      "tareas": [6.0, 6.5, 6.2], "examen": 5.9},
        {"nombre": "Hernán Pinto",       "tareas": [5.0, 4.8, 5.2], "examen": 5.5},
        {"nombre": "Isidora Reyes",      "tareas": [7.0, 7.0, 6.8], "examen": 7.0},
        {"nombre": "Jorge Espinoza",     "tareas": [4.2, 3.8, 4.5], "examen": 4.0},
        {"nombre": "Karla Ibáñez",       "tareas": [5.8, 6.2, 5.5], "examen": 6.0},
    ],
}

print("INFORME FINAL — PROGRAMACIÓN EN PYTHON")
print("Universidad Adolfo Ibáñez — Negocios y Tecnología")

for seccion, alumnos in secciones.items():
    generar_informe_seccion(seccion, alumnos)

INFORME FINAL — PROGRAMACIÓN EN PYTHON
Universidad Adolfo Ibáñez — Negocios y Tecnología

  SECCIÓN: Sección 01 — Lunes/Miércoles
Alumno                     T1    T2    T3  Examen   Prom Estado
--------------------------------------------------------------------
  Alejandro Muñoz          6.5   7.0   5.8     6.2    6.4 ✅ Muy Bueno
  Beatriz Soto             4.0   3.5   4.5     3.8    3.9 🔴 Insuficiente
  Camila Fuentes           5.5   6.0   5.0     5.8    5.6 ✅ Muy Bueno
  Diego Contreras          7.0   6.8   7.0     6.9    6.9 ⭐ Destacado
  Elena Vásquez            4.5   5.0   4.8     4.2    4.7 🟢 Bueno
  Felipe Araya             3.0   2.5   3.5     3.0    3.1 🔴 Insuficiente
--------------------------------------------------------------------
  Promedio sección:   5.10
  Aprobados:          4/6 (67%)
  Nota máxima:        6.94
  Nota mínima:        3.10

  SECCIÓN: Sección 02 — Martes/Jueves
Alumno                     T1    T2    T3  Examen   Prom Estado
------------------------------

## 6.4 Proyecto Final: Tu Propio Sistema

Como proyecto de cierre, construirás un **sistema completo** que integre todos los conceptos del curso. Tu programa debe incluir:

### Requisitos mínimos

**1. Entrada de datos** (al menos uno de los siguientes):
- Lectura desde archivo CSV
- Datos ingresados por el usuario con `input()`
- Datos generados dinámicamente (con `random`)

**2. Procesamiento** (todos los siguientes):
- Al menos 5 funciones propias
- Control de flujo con `if/elif/else`
- Al menos un bucle `for` y un bucle `while`
- Uso de al menos 2 estructuras de datos (lista, dict, tupla o set)
- Manejo de errores con `try/except`

**3. Salida estructurada** (al menos uno de los siguientes):
- Reporte formateado en pantalla
- Exportación a archivo CSV
- Menú interactivo

### Ideas de proyectos

| Proyecto | Descripción |
|---------|-------------|
| Sistema de Biblioteca | Gestión de préstamos de libros |
| Tienda Online | Carrito de compras con descuentos y boleta |
| Control de Asistencia | Registro y reporte de asistencia de empleados |
| Análisis Financiero | Presupuesto vs gasto real con alertas |
| Sistema de Encuestas | Registro y análisis de respuestas |
| Gestión de Tareas | Lista de pendientes con prioridades |

:::{note}
El proyecto debe ser entregado como un Jupyter Notebook con todas las celdas ejecutadas y explicaciones en celdas de texto que describan las decisiones de diseño.
:::


## Resumen del Curso

¡Felicitaciones por completar el curso de Programación en Python!

| Unidad | Tema | Conceptos clave |
|--------|------|----------------|
| 1 | Resolución de Problemas | Algoritmos, pseudocódigo, descomposición |
| 2 | Introducción a Python | Variables, tipos, operadores, print, input |
| 3 | Control de Flujo | if/elif/else, for, while, break, continue |
| 4 | Funciones | def, return, scope, try/except |
| 5 | Estructuras de Datos | Lista, dict, tupla, set, CSV |
| 6 | Integración | Programas completos, casos empresariales |

### Próximos pasos sugeridos

- **Pandas**: análisis de datos tabulares a escala
- **Matplotlib / Seaborn**: visualización de datos
- **Flask / FastAPI**: desarrollo de APIs web
- **SQLite / PostgreSQL**: manejo de bases de datos
- **NumPy**: computación científica

:::{tip}
La mejor forma de seguir aprendiendo es **construir proyectos propios** que resuelvan problemas que te interesen. ¡El código que escribes para resolver tus propios problemas es el que mejor recuerdas!
:::

---
*Apunte desarrollado para el curso Programación en Python — Carrera Negocios y Tecnología, Universidad Adolfo Ibáñez.*
